In [ ]:
import sys
import glob
from datetime import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.interpolate import interp1d
import statsmodels.api as sm

import matplotlib.pyplot as plt
from ctapipe.io import EventSource
from ctapipe.image import ImageProcessor, tailcuts_clean
from ctapipe.calib import CameraCalibrator
from ctapipe.visualization import CameraDisplay
from astropy.table import Table, vstack

#from muonpipe import usefull_func
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd



In [ ]:
filename = '/Users/vdk/muons2024//simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
camgeom = source.subarray.tel[1].camera.geometry

start_date_2019 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397

# Simulations load (standard config, nsb and mirror reflectivity tuned to the 2024 year)

## `mirror_align_random` = 0.0043 deg (Reference value)

In [ ]:
listdir1 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/nsbtune2024year_no_outliers_additional_true/tables/*')
listdir = listdir1 + listdir2

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    
    # Check if 'good_ring' exists and convert to bool
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_00043_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]
df_sim_00043_alignment

## `mirror_align_random` = 0.0025 deg

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_00025alignment/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_00025_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]
df_sim_00025_alignment

## `mirror_align_random` = 0.0092 deg

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_00092alignment/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_0092_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]
df_sim_0092_alignment

## `mirror_align_random` = 0.01 deg

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_001alignment/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_001_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]
df_sim_001_alignment

## `mirror_align_random` = 0.015 deg

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_0015alignment/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_0015_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_sim_0015_alignment

## `mirror_align_random` = 0.02 deg

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_002alignment/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_002_alignment = df[
     df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]
df_sim_002_alignment

# Data load

In [ ]:
muon_files = glob.glob('/Users/vdk/Documents/DocumentsVadymMacBookWork/all_muon_fits/low_nsb_filter/*')
for i, filename in enumerate(sorted(muon_files)):
    print(i, filename)

In [ ]:
df_files = [
    pd.read_csv(muon_file, na_values=['NA', '?']).query(
        '(muon_efficiency < @muon_efficiency_cut) & '
        '(size_outside < @size_outside_cut) & '
        '(event_time >= @start_date_2024) & '
        '(event_time <= @end_date_2024)'
    )
    for muon_file in sorted(muon_files)[10:]
]

df_good_data_real = pd.concat(df_files, ignore_index=True)
df_good_data_real

# Applying necessary cuts on the datasets

### Cuts values

In [ ]:
ring_completeness_cut = 0.8
ring_containment_cut = 0
min_radius_cut = 1.05
max_radius_cut = 1.15


In [ ]:
def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

df_sim_00025_cut = df_sim_00025_alignment[
    (df_sim_00025_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_00025_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_00025_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_00025_alignment['ring_radius'] < max_radius_cut)
]
print(f"Events survived for df_sim_00025_cut: {len(df_sim_00025_cut)} ({calculate_survival_percentage(df_sim_00025_alignment, df_sim_00025_cut):.2f}%)")

df_sim_00043_cut = df_sim_00043_alignment[
    (df_sim_00043_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_00043_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_00043_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_00043_alignment['ring_radius'] < max_radius_cut)
]
print(f"Events survived for df_sim_00043_cut: {len(df_sim_00043_cut)} ({calculate_survival_percentage(df_sim_00043_alignment, df_sim_00043_cut):.2f}%)")

df_sim_0092_cut = df_sim_0092_alignment[
    (df_sim_0092_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_0092_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_0092_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_0092_alignment['ring_radius'] < max_radius_cut)
]
print(f"Events survived for df_sim_0092_cut: {len(df_sim_0092_cut)} ({calculate_survival_percentage(df_sim_0092_alignment, df_sim_0092_cut):.2f}%)")

df_sim_001_cut = df_sim_001_alignment[
    (df_sim_001_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_001_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_001_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_001_alignment['ring_radius'] < max_radius_cut)
]
print(f"Events survived for df_sim_001_cut: {len(df_sim_001_cut)} ({calculate_survival_percentage(df_sim_001_alignment, df_sim_001_cut):.2f}%)")

df_sim_0015_cut = df_sim_0015_alignment[
    (df_sim_0015_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_0015_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_0015_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_0015_alignment['ring_radius'] < max_radius_cut)
]
print(f"Events survived for df_sim_0015_cut: {len(df_sim_0015_cut)} ({calculate_survival_percentage(df_sim_0015_alignment, df_sim_0015_cut):.2f}%)")

df_sim_002_cut = df_sim_002_alignment[
    (df_sim_002_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_002_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_002_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_002_alignment['ring_radius'] < max_radius_cut)
]
print(f"Events survived for df_sim_002_cut: {len(df_sim_002_cut)} ({calculate_survival_percentage(df_sim_002_alignment, df_sim_002_cut):.2f}%)")

df_data_cut = df_good_data_real[
    (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
    (df_good_data_real['ring_containment'] > ring_containment_cut) &
    (df_good_data_real['ring_radius'] > min_radius_cut) &
    (df_good_data_real['ring_radius'] < max_radius_cut)
]
print(f"Events survived for df_data_cut: {len(df_data_cut)} ({calculate_survival_percentage(df_good_data_real, df_data_cut):.2f}%)")

# Ring width distribution

In [ ]:
bin_number = 600
plt.figure(figsize=(10, 8))

# Calculate and print mean and std for each data frame
print(f"Real Data from 2024 year - Mean: {df_data_cut['ring_width'].mean():.3f}, Std: {df_data_cut['ring_width'].std():.3f}")
print(f"0.0043 mirror_alignment - Mean: {df_sim_00043_cut['ring_width'].mean():.3f}, Std: {df_sim_00043_cut['ring_width'].std():.3f}")
print(f"0.015 mirror_alignment - Mean: {df_sim_0015_cut['ring_width'].mean():.3f}, Std: {df_sim_0015_cut['ring_width'].std():.3f}")

mean_data = df_data_cut['ring_width'].mean()
std_data = df_data_cut['ring_width'].std()
mean_0043 = df_sim_00043_cut['ring_width'].mean()
std_0043 = df_sim_00043_cut['ring_width'].std()
mean_0015 = df_sim_0015_cut['ring_width'].mean()
std_0015 = df_sim_0015_cut['ring_width'].std()

# Print mean and std on the figure
plt.text(0.125, 15, f"Mean: {mean_data:.3f}, Std: {std_data:.3f}", fontsize=12, color='blue')
plt.text(0.125, 14, f"Mean: {mean_0043:.3f}, Std: {std_0043:.3f}", fontsize=12, color='orange')
plt.text(0.125, 13, f"Mean: {mean_0015:.3f}, Std: {std_0015:.3f}", fontsize=12, color='green')


# Plot histograms
plt.hist(df_data_cut['ring_width'], bins=bin_number, alpha=0.8, label='Real Data 2024 year', density=True, histtype='step', linewidth=3)
plt.hist(df_sim_00043_cut['ring_width'], bins=bin_number, alpha=0.8, label='Sim random_mirror_alignment = 0.0043 [deg]', density=True, histtype='step', linewidth=3)
plt.hist(df_sim_0015_cut['ring_width'], bins=bin_number, alpha=0.8, label='Sim random_mirror_alignment = 0.015 [deg]', density=True, histtype='step', linewidth=3)

# Add vertical lines for the 0.2 quantile
plt.axvline(df_data_cut['ring_width'].quantile(0.2), color='blue', linestyle='dashed', linewidth=2, label='0.2 Quantile Real Data')
plt.axvline(df_sim_00043_cut['ring_width'].quantile(0.2), color='orange', linestyle='dashed', linewidth=2, label='0.2 Quantile Sim 0.0043 deg')
plt.axvline(df_sim_0015_cut['ring_width'].quantile(0.2), color='green', linestyle='dashed', linewidth=2, label='0.2 Quantile Sim 0.015 deg')

# Customize plot labels and limits
plt.xlabel('Ring Width [deg]', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.title(f'rings with completeness > {ring_completeness_cut}, radius between {min_radius_cut, max_radius_cut}', fontsize=20)
plt.legend()
plt.xlim(0, 0.2)

# Show the plot

#plt.savefig(f'/Users/vdk/muons2024/psf_work/ring_radius_cut_completeness_scan/2024year_width_hist_completeness>{ring_completeness_cut}_radius_between{min_radius_cut,max_radius_cut}.png', dpi=200)
plt.show()

# (Ring width / radius) vs radius

In [ ]:
# Define the number of entries to use from each dataset
MAX_ENTRIES = 20000
NUM_BINS = 10
COMPLETENESS_CUT = ring_completeness_cut
QUANTILE_CUT = 0.2

# Calculate the 20th percentile (quantile 0.2) for each dataset
quantiles = {
    'sim_00025_alignment': df_sim_00025_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_2024_tune': df_sim_00043_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_0092_alignment': df_sim_0092_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_001_alignment': df_sim_001_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_0015_alignment': df_sim_0015_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_002_alignment': df_sim_002_cut['ring_width'].quantile(QUANTILE_CUT),
    'data': df_data_cut['ring_width'].quantile(QUANTILE_CUT)
}

# Define a list of tuples containing each dataframe, its corresponding quantile, and label
datasets = [
    (df_sim_00025_cut[df_sim_00025_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_00025_alignment'], '0.0025 deg'),
    (df_sim_00043_cut[df_sim_00043_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_2024_tune'], '0.0043 deg (Reference)'),
    (df_sim_0092_cut[df_sim_0092_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_0092_alignment'], '0.0092 deg'),
    (df_sim_001_cut[df_sim_001_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_001_alignment'], '0.01 deg'),
    (df_sim_0015_cut[df_sim_0015_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_0015_alignment'], '0.015 deg'),
    (df_sim_002_cut[df_sim_002_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_002_alignment'], '0.02 deg'),
    (df_data_cut[df_data_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['data'], 'Data')
]

datasets = [
    (df_sim_00043_cut[df_sim_00043_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_2024_tune'], '0.0043 deg (Reference)'),
    (df_sim_001_cut[df_sim_001_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_001_alignment'], '0.01 deg'),
    (df_sim_0015_cut[df_sim_0015_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_0015_alignment'], '0.015 deg'),
    (df_data_cut[df_data_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['data'], 'Data')
]

# Initialize a dictionary to store binned data for each dataset
binned_data = {}

# Iterate over each dataset to preprocess data
for df, quantile, label in datasets:
    # Limit to the first MAX_ENTRIES entries
    limited_df = df.iloc[:MAX_ENTRIES]
    
    # Filter the data based on the quantile
    filtered_df = limited_df[limited_df['ring_width'] < quantile].copy()
    
    # Check if filtered_df is not empty to avoid errors
    if filtered_df.empty:
        print(f"Warning: No data points below the 20th percentile for '{label}' in the first {MAX_ENTRIES} entries.")
        continue
    
    # Bin 'ring_radius' into NUM_BINS equal-width bins
    filtered_df['radius_bin'] = pd.cut(filtered_df['ring_radius'], bins=NUM_BINS, labels=False)
    
    # Remove any rows with NaN in 'radius_bin' (can occur if 'ring_radius' has values outside the bin range)
    filtered_df = filtered_df.dropna(subset=['radius_bin'])
    
    # Convert 'radius_bin' to integer
    filtered_df['radius_bin'] = filtered_df['radius_bin'].astype(int)
    
    # Group by 'radius_bin' and compute mean 'ring_radius', 'ring_width', and the ratio 'ring_width'/'ring_radius'
    grouped = filtered_df.groupby('radius_bin').agg({
        'ring_radius': 'mean',
        'ring_width': 'mean'
    }).reset_index()
    grouped['width_radius_ratio'] = grouped['ring_width'] / grouped['ring_radius']
    
    # Store the grouped data
    binned_data[label] = grouped

# Plot settings
alpha_reg = 0.7
palette = sns.color_palette("husl", len(binned_data))[:-1] + ['k']

plt.figure(figsize=(12, 10))
flag = '2023-24'

# Iterate over each binned dataset and plot with regression lines and confidence intervals
for i, (label, grouped_df) in enumerate(binned_data.items()):
    sns.regplot(
        x='ring_radius',
        y='width_radius_ratio',
        data=grouped_df,
        label=label,
        scatter_kws={'alpha': alpha_reg, 's': 100},
        line_kws={'alpha': alpha_reg, 'color': palette[i]},
        ci=95,
        color=palette[i],
        robust=True
    )

# Customize plot labels and limits
plt.xlabel('Muon Radius [deg]', fontsize=24)
plt.ylabel('Muon Ring Width / Radius', fontsize=24)
plt.ylim(0.03, 0.06)
plt.xlim(min_radius_cut, max_radius_cut)

# Enhance the legend
plt.legend(prop={'size': 26}, loc='upper right', frameon=True)

# Add a title for context
#plt.title(f'Ring width vs radius ratio for 2024 year, {QUANTILE_CUT*100}% quantile, completeness > {COMPLETENESS_CUT}', fontsize=18)

# Improve layout and display the plot
plt.tight_layout()
plt.grid(alpha=0.3)

# Optionally, save the figure
#plt.savefig(f'/Users/vdk/muons2024/psf_work/width_radius/widthradius_vs_radius_completeness_{COMPLETENESS_CUT}_quantile{QUANTILE_CUT*100}%_nsb_tune2024.pdf', dpi=200)
plt.show()

# Ring radius histogram

In [ ]:
bin_number = 50
plt.figure(figsize=(12, 10))
plt.xlim(0.9,1.3)
z,x,c = plt.hist(df_good_data_real['ring_radius'], bins=bin_number, alpha=0.8, label='Real Data 2024 year', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_00043_alignment['ring_radius'], bins=bin_number, alpha=0.8, label='0.0043 deg', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_001_alignment['ring_radius'], bins=bin_number, alpha=0.8, label='0.01 deg', density=True, histtype='step', linewidth=3)

plt.legend(prop={'size': 20})
plt.xlabel('Ring Radius [deg]', fontsize=20)
plt.ylabel('Frequency', fontsize=20)

In [ ]:
bin_number = 40
plt.figure(figsize=(12, 10))
z,x,c = plt.hist(df_data_cut['ring_radius'], bins=bin_number, alpha=0.8, label='Real Data 2024 year', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_00043_cut['ring_radius'], bins=bin_number, alpha=0.8, label='0.0043 deg', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_001_cut['ring_radius'], bins=bin_number, alpha=0.8, label='0.01 deg', density=True, histtype='step', linewidth=3)

plt.legend(prop={'size': 20})
plt.xlim(1,1.17)
plt.xlabel('Ring Radius [deg]', fontsize=20)
plt.ylabel('Frequency', fontsize=20)

In [ ]:
bin_number = 40
plt.figure(figsize=(12, 10))
z,x,c = plt.hist(df_data_cut['ring_radius'], bins=bin_number, alpha=0.8, label='Real Data 2024 year', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_00043_cut['ring_radius'], bins=bin_number, alpha=0.8, label='0.0043 deg', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_001_cut['ring_radius'], bins=bin_number, alpha=0.8, label='0.01 deg', density=True, histtype='step', linewidth=3)

plt.legend(prop={'size': 20})
plt.xlim(1,1.15)
plt.xlabel('Ring Radius [deg]', fontsize=20)
plt.ylabel('Frequency', fontsize=20)

# Cut on the impact parameter

## Cuts values

In [ ]:
ring_completeness_cut = 0.6
ring_containment_cut = 0
min_radius_cut = 1.06
max_radius_cut = 1.15

min_impact_cut = 0
max_impact_cut = 3

In [ ]:
def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

df_sim_00025_cut = df_sim_00025_alignment[
    (df_sim_00025_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_00025_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_00025_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_00025_alignment['ring_radius'] < max_radius_cut) &
    (df_sim_00025_alignment['impact_parameter'] > min_impact_cut) &
    (df_sim_00025_alignment['impact_parameter'] < max_impact_cut)
]
print(f"Events survived for df_sim_00025_cut: {len(df_sim_00025_cut)} ({calculate_survival_percentage(df_sim_00025_alignment, df_sim_00025_cut):.2f}%)")

df_sim_00043_cut = df_sim_00043_alignment[
    (df_sim_00043_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_00043_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_00043_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_00043_alignment['ring_radius'] < max_radius_cut) &
    (df_sim_00043_alignment['impact_parameter'] > min_impact_cut) &
    (df_sim_00043_alignment['impact_parameter'] < max_impact_cut)
]
print(f"Events survived for df_sim_00043_cut: {len(df_sim_00043_cut)} ({calculate_survival_percentage(df_sim_00043_alignment, df_sim_00043_cut):.2f}%)")

df_sim_0092_cut = df_sim_0092_alignment[
    (df_sim_0092_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_0092_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_0092_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_0092_alignment['ring_radius'] < max_radius_cut) &
    (df_sim_0092_alignment['impact_parameter'] > min_impact_cut) &
    (df_sim_0092_alignment['impact_parameter'] < max_impact_cut)
]
print(f"Events survived for df_sim_0092_cut: {len(df_sim_0092_cut)} ({calculate_survival_percentage(df_sim_0092_alignment, df_sim_0092_cut):.2f}%)")

df_sim_001_cut = df_sim_001_alignment[
    (df_sim_001_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_001_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_001_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_001_alignment['ring_radius'] < max_radius_cut) &
    (df_sim_001_alignment['impact_parameter'] > min_impact_cut) &
    (df_sim_001_alignment['impact_parameter'] < max_impact_cut)
]
print(f"Events survived for df_sim_001_cut: {len(df_sim_001_cut)} ({calculate_survival_percentage(df_sim_001_alignment, df_sim_001_cut):.2f}%)")

df_sim_0015_cut = df_sim_0015_alignment[
    (df_sim_0015_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_0015_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_0015_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_0015_alignment['ring_radius'] < max_radius_cut) &
    (df_sim_0015_alignment['impact_parameter'] > min_impact_cut) &
    (df_sim_0015_alignment['impact_parameter'] < max_impact_cut)
]
print(f"Events survived for df_sim_0015_cut: {len(df_sim_0015_cut)} ({calculate_survival_percentage(df_sim_0015_alignment, df_sim_0015_cut):.2f}%)")

df_sim_002_cut = df_sim_002_alignment[
    (df_sim_002_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_002_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_002_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_002_alignment['ring_radius'] < max_radius_cut) &
    (df_sim_002_alignment['impact_parameter'] > min_impact_cut) &
    (df_sim_002_alignment['impact_parameter'] < max_impact_cut)
]
print(f"Events survived for df_sim_002_cut: {len(df_sim_002_cut)} ({calculate_survival_percentage(df_sim_002_alignment, df_sim_002_cut):.2f}%)")

df_data_cut = df_good_data_real[
    (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
    (df_good_data_real['ring_containment'] > ring_containment_cut) &
    (df_good_data_real['ring_radius'] > min_radius_cut) &
    (df_good_data_real['ring_radius'] < max_radius_cut) &
    (df_good_data_real['impact_parameter'] > min_impact_cut) &
    (df_good_data_real['impact_parameter'] < max_impact_cut)
]
print(f"Events survived for df_data_cut: {len(df_data_cut)} ({calculate_survival_percentage(df_good_data_real, df_data_cut):.2f}%)")

In [ ]:
bin_number = 200
plt.figure(figsize=(10, 8))

# Calculate and print mean and std for each data frame
print(f"Real Data from 2024 year - Mean: {df_data_cut['ring_width'].mean():.3f}, Std: {df_data_cut['ring_width'].std():.3f}")
print(f"0.0043 mirror_alignment - Mean: {df_sim_00043_cut['ring_width'].mean():.3f}, Std: {df_sim_00043_cut['ring_width'].std():.3f}")
print(f"0.015 mirror_alignment - Mean: {df_sim_0015_cut['ring_width'].mean():.3f}, Std: {df_sim_0015_cut['ring_width'].std():.3f}")

mean_data = df_data_cut['ring_width'].mean()
std_data = df_data_cut['ring_width'].std()
mean_0043 = df_sim_00043_cut['ring_width'].mean()
std_0043 = df_sim_00043_cut['ring_width'].std()
mean_0015 = df_sim_0015_cut['ring_width'].mean()
std_0015 = df_sim_0015_cut['ring_width'].std()

# Print mean and std on the figure
plt.text(0.125, 15, f"Mean: {mean_data:.3f}, Std: {std_data:.3f}", fontsize=12, color='blue')
plt.text(0.125, 14, f"Mean: {mean_0043:.3f}, Std: {std_0043:.3f}", fontsize=12, color='orange')
plt.text(0.125, 13, f"Mean: {mean_0015:.3f}, Std: {std_0015:.3f}", fontsize=12, color='green')


# Plot histograms
plt.hist(df_data_cut['ring_width'], bins=bin_number, alpha=0.8, label='Real Data 2024 year', density=True, histtype='step', linewidth=3)
plt.hist(df_sim_00043_cut['ring_width'], bins=bin_number, alpha=0.8, label='Sim random_mirror_alignment = 0.0043 [deg]', density=True, histtype='step', linewidth=3)
plt.hist(df_sim_0015_cut['ring_width'], bins=bin_number, alpha=0.8, label='Sim random_mirror_alignment = 0.015 [deg]', density=True, histtype='step', linewidth=3)

# Add vertical lines for the 0.2 quantile
plt.axvline(df_data_cut['ring_width'].quantile(0.2), color='blue', linestyle='dashed', linewidth=2, label='0.2 Quantile Real Data')
plt.axvline(df_sim_00043_cut['ring_width'].quantile(0.2), color='orange', linestyle='dashed', linewidth=2, label='0.2 Quantile Sim 0.0043 deg')
plt.axvline(df_sim_0015_cut['ring_width'].quantile(0.2), color='green', linestyle='dashed', linewidth=2, label='0.2 Quantile Sim 0.015 deg')

# Customize plot labels and limits
plt.xlabel('Ring Width [deg]', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.title(f'rings with completeness > {ring_completeness_cut}, radius between {min_radius_cut, max_radius_cut}', fontsize=20)
plt.legend()
plt.xlim(0, 0.2)

# Show the plot

#plt.savefig(f'/Users/vdk/muons2024/psf_work/ring_radius_cut_completeness_scan/2024year_width_hist_completeness>{ring_completeness_cut}_radius_between{min_radius_cut,max_radius_cut}.png', dpi=200)
plt.show()

In [ ]:
# Define the number of entries to use from each dataset
MAX_ENTRIES = 10000
NUM_BINS = 10
COMPLETENESS_CUT = ring_completeness_cut
QUANTILE_CUT = 0.2

# Calculate the 20th percentile (quantile 0.2) for each dataset
quantiles = {
    'sim_00025_alignment': df_sim_00025_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_2024_tune': df_sim_00043_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_0092_alignment': df_sim_0092_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_001_alignment': df_sim_001_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_0015_alignment': df_sim_0015_cut['ring_width'].quantile(QUANTILE_CUT),
    'sim_002_alignment': df_sim_002_cut['ring_width'].quantile(QUANTILE_CUT),
    'data': df_data_cut['ring_width'].quantile(QUANTILE_CUT)
}

# Define a list of tuples containing each dataframe, its corresponding quantile, and label
# datasets = [
#     (df_sim_00025_cut[df_sim_00025_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_00025_alignment'], '0.0025 deg'),
#     (df_sim_00043_cut[df_sim_00043_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_2024_tune'], '0.0043 deg (Reference)'),
#     (df_sim_0092_cut[df_sim_0092_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_0092_alignment'], '0.0092 deg'),
#     (df_sim_001_cut[df_sim_001_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_001_alignment'], '0.01 deg'),
#     (df_sim_0015_cut[df_sim_0015_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_0015_alignment'], '0.015 deg'),
#     (df_sim_002_cut[df_sim_002_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_002_alignment'], '0.02 deg'),
#     (df_data_cut[df_data_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['data'], 'Data')
# ]

datasets = [
    (df_sim_00043_cut[df_sim_00043_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_2024_tune'], '0.0043 deg (Reference)'),
    (df_sim_001_cut[df_sim_001_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_001_alignment'], '0.01 deg'),
    (df_sim_0015_cut[df_sim_0015_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['sim_0015_alignment'], '0.015 deg'),
    (df_data_cut[df_data_cut['ring_completeness'] > COMPLETENESS_CUT], quantiles['data'], 'Data')
]

# Initialize a dictionary to store binned data for each dataset
binned_data = {}

# Iterate over each dataset to preprocess data
for df, quantile, label in datasets:
    # Limit to the first MAX_ENTRIES entries
    limited_df = df.iloc[:MAX_ENTRIES]
    
    # Filter the data based on the quantile
    filtered_df = limited_df[limited_df['ring_width'] < quantile].copy()
    
    # Check if filtered_df is not empty to avoid errors
    if filtered_df.empty:
        print(f"Warning: No data points below the 20th percentile for '{label}' in the first {MAX_ENTRIES} entries.")
        continue
    
    # Bin 'ring_radius' into NUM_BINS equal-width bins
    filtered_df['radius_bin'] = pd.cut(filtered_df['ring_radius'], bins=NUM_BINS, labels=False)
    
    # Remove any rows with NaN in 'radius_bin' (can occur if 'ring_radius' has values outside the bin range)
    filtered_df = filtered_df.dropna(subset=['radius_bin'])
    
    # Convert 'radius_bin' to integer
    filtered_df['radius_bin'] = filtered_df['radius_bin'].astype(int)
    
    # Group by 'radius_bin' and compute mean 'ring_radius', 'ring_width', and the ratio 'ring_width'/'ring_radius'
    grouped = filtered_df.groupby('radius_bin').agg({
        'ring_radius': 'mean',
        'ring_width': 'mean'
    }).reset_index()
    grouped['width_radius_ratio'] = grouped['ring_width'] / grouped['ring_radius']
    
    # Store the grouped data
    binned_data[label] = grouped

# Plot settings
alpha_reg = 0.7
palette = sns.color_palette("husl", len(binned_data))[:-1] + ['k']

plt.figure(figsize=(12, 10))
flag = '2023-24'

# Iterate over each binned dataset and plot with regression lines and confidence intervals
for i, (label, grouped_df) in enumerate(binned_data.items()):
    sns.regplot(
        x='ring_radius',
        y='width_radius_ratio',
        data=grouped_df,
        label=label,
        scatter_kws={'alpha': alpha_reg, 's': 100},
        line_kws={'alpha': alpha_reg, 'color': palette[i]},
        ci=95,
        color=palette[i],
        robust=True
    )

# Customize plot labels and limits
plt.xlabel('Muon Radius [deg]', fontsize=24)
plt.ylabel('Muon Ring Width / Radius', fontsize=24)
plt.ylim(0.03, 0.06)
plt.xlim(min_radius_cut, max_radius_cut)

# Enhance the legend
plt.legend(prop={'size': 26}, loc='upper right', frameon=True)

# Add a title for context
#plt.title(f'Ring width vs radius ratio for 2024 year, {QUANTILE_CUT*100}% quantile, completeness > {COMPLETENESS_CUT}', fontsize=18)

# Improve layout and display the plot
plt.tight_layout()
plt.grid(alpha=0.3)

# Optionally, save the figure
#plt.savefig(f'/Users/vdk/muons2024/psf_work/width_radius/widthradius_vs_radius_completeness_{COMPLETENESS_CUT}_quantile{QUANTILE_CUT*100}%_nsb_tune2024.pdf', dpi=200)
plt.show()

In [ ]:
bin_number = 40
plt.figure(figsize=(12, 10))
z,x,c = plt.hist(df_data_cut['ring_radius'], bins=bin_number, alpha=0.8, label='Real Data 2024 year', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_00043_cut['ring_radius'], bins=bin_number, alpha=0.8, label='0.0043 deg', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_001_cut['ring_radius'], bins=bin_number, alpha=0.8, label='0.01 deg', density=True, histtype='step', linewidth=3)

plt.legend(prop={'size': 20})
plt.xlim(1,1.17)
plt.xlabel('Ring Radius [deg]', fontsize=20)
plt.ylabel('Frequency', fontsize=20)

In [ ]:
bin_number = 50
plt.figure(figsize=(12, 10))
plt.xlim(0.9,1.3)
z,x,c = plt.hist(df_good_data_real['ring_radius'], bins=bin_number, alpha=0.8, label='Real Data 2024 year', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_00043_alignment['ring_radius'], bins=bin_number, alpha=0.8, label='0.0043 deg', density=True, histtype='step', linewidth=3)
z,x,c = plt.hist(df_sim_001_alignment['ring_radius'], bins=bin_number, alpha=0.8, label='0.01 deg', density=True, histtype='step', linewidth=3)

plt.legend(prop={'size': 20})
plt.xlabel('Ring Radius [deg]', fontsize=20)
plt.ylabel('Frequency', fontsize=20)